In [1]:
# 'requests' module does not fit in this case 'cause of anti-bot cloudflare protection
# instead, use cloudscraper (https://pypi.org/project/cloudscraper/)
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time, pyautogui, cloudscraper, json
from ipywidgets import IntProgress
from IPython.display import display

## Useful functions

In [3]:
def get_part_of_the_content(page_source, result):
    prev = len(result)
    soup = BeautifulSoup(page_source, 'html.parser')
    res = soup.find_all(name = 'a', attrs = {'class': 'olx-ad-card__link-wrapper', 'data-ds-component': 'DS-NewAdCard-Link'})
    out = [link['href'] for link in res if link['href'] not in result]
    result.extend(out)
    return prev, len(result)

def scrape_page(url):
    scraper = cloudscraper.create_scraper()
    page = scraper.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    r = {}
    r['url'] = url
    
    try:
        r['titulo_anuncio'] = soup.find(name = 'h1', attrs = {'class': 'ad__sc-45jt43-0 htAiPK sc-VigVT ZEEcn', 'data-ds-component': 'DS-Text'}).text
    except:
        r['titulo_anuncio'] = ''
        
    try:
        aux = soup.find(name = 'span', attrs = {'class': 'ad__sc-1oq8jzc-0 dWayMW sc-VigVT dHNWJq', 'data-ds-component': 'DS-Text'}).text
        aux2 = aux.split(' ')
        r['data'] = aux2[2]
        r['hora'] = aux2[-1]
    except:
        r['data'] = ''
        r['hora'] = ''
    
    try:
        r['codigo'] = soup.find(name = 'span', attrs = {'class': 'ad__sc-16iz3i7-0 hjLLUR sc-VigVT dHNWJq', 'data-ds-component': 'DS-Text'}).text.split(' ')[-1]
    except:
        r['codigo'] = ''
        
    try:
        r['preco'] = soup.find(name = 'h2', attrs = {'class': 'ad__sc-n916xm-3 caQRQy sc-VigVT honhBH', 'data-ds-component': 'DS-Text'}).text.split(' ')[-1]
    except:
        r['preco'] = ''
        
    try:
        aux = soup.find_all(name = 'span', attrs = {'class': 'ad__sc-2iplj6-3 xIkHt sc-VigVT rycv', 'data-ds-component': 'DS-Text'})    
        r['valor_condominio'] = aux[0].text
        r['valor_iptu'] = aux[1].text
    except:
        r['valor_condominio'] = ''
        r['valor_iptu'] = ''
        
    try:
        r['descricao'] = soup.find(name = 'span', attrs = {'class': 'ad__sc-1sj3nln-1 fMgwdS sc-VigVT jcyavR', 'data-ds-component': 'DS-Text'}).text
    except:
        r['descricao'] = ''
    
    aux = soup.find(name = 'div', attrs = {'class': 'ad__sc-10rz2dk-2 fxAfax'})
    try:           
        aux2 = [i.text for i in aux.find_all('p')]
    except:
        r['endereco'] = ''
        r['bairro'] = ''
        r['cidade'] = ''
        r['estado'] = ''
        r['cep'] = ''
    else:
        r['endereco'] = aux2[0]
        
        aux3 = aux2[1].split('-')
        if len(aux3) > 2:
            r['bairro'] = '-'.join(aux3[:-1]).strip()
        else:
            r['bairro'] = aux3[0]

        aux4 = aux3[-1].split(',')    
        r['cidade'] = aux4[0].strip()    
        r['estado'] = aux4[1].strip()    
        r['cep'] = aux2[2].split(' ')[-1]    
    
    aux = soup.find_all(name = 'div', attrs = {'class': 'ad__sc-1yh67o8-2 dUSUNf'})
    try:        
        r['quartos'] = aux[0].find('a').text
    except:
        r['quartos'] = ''
    
    try:
        r['areas'] = aux[1].find('p').text
    except:
        r['areas'] = ''
    
    try:
        r['vagas_carros'] = aux[2].find('p').text
    except:
        r['vagas_carros'] = ''
    
    try:
        r['banheiros'] = aux[3].find('p').text
    except:
        r['banheiros'] = ''
        
    aux = soup.find_all(name = 'div', attrs = {'class': 'ad__sc-1x4pdhw-1 deKSko'})
    try:
        r['car_imovel'] = [i.text for i in aux[0]]        
    except:
        r['car_imovel'] = []
    
    try:
        r['car_condominio'] = [i.text for i in aux[1]]
    except:
        r['car_condominio'] = []
    
    aux = soup.find(name = 'script', attrs = {'id': 'initial-data'})
    aux = json.loads(aux['data-json'])
    
    
    try:
        cep = aux['ad']['location']['zipcode']
        signature = aux['ad']['signedPostalCode']
        
        search_pos = f'https://busca-cep.olx.com.br/geocode?cep={cep}&signature={signature}'
        page = scraper.get(search_pos)
        aux2 = json.loads(page.text)
        r['lat'] = aux2['lat']
        r['lon'] = aux2['lng']
    except:
        r['lat'] = ''
        r['lon'] = ''
        
    r['json'] = aux
    
    return r

## Get links with 'Speedy Gonzales' robot

In [ ]:
# You are not supposed to change these constant values
olx_url = 'https://www.olx.com.br/imoveis/venda/estado-pa/regiao-de-belem/belem'
width = height = 800
scroll = 1500
driver = webdriver.Firefox()
links = []

# Position of the screen (not required)
ss_w, ss_h = pyautogui.size()
driver.set_window_size(width, height)
driver.set_window_position(ss_w / 2 - width / 2, ss_h / 2 - height / 2)

# Change according to what you want to grab
# You will have around links_per_page * len(pages) urls
links_per_page = 10 # ads per page
pages = list(range(1, 6)) # pages to iterate


f = IntProgress(min = 0, max = len(pages), description = 'Progress:')
display(f) # display the bar

for page_number in pages:
    pixels_h = scroll
    counter = 0
    driver.get(olx_url + f'?o={page_number}')
    # print(f'Page {page_number}:')
    # print(f'---Total links: ', end = ' ')
    
    while len(links) < links_per_page * page_number and counter < 2:
        driver.execute_script(f'window.scrollTo(0, {pixels_h});')
        time.sleep(1)
        prev, act = get_part_of_the_content(driver.page_source, links)
        pixels_h += scroll        
        # print(f'{len(links)}', end = ' ')
        if prev == act:
            counter += 1
    f.value += 1
    # print('')
driver.quit()

## Scrape data from each ad page

In [ ]:
ads = []

f = IntProgress(min = 0, max = len(links), description = 'Progress:')
display(f) # display the bar

for l in links:
    ads.append(scrape_page(l))
    f.value += 1